In [1]:
import tensorflow as tf
import math
import import_ipynb
import skipgram_batch_generator
import numpy as np

C:\Users\Fahimeh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


importing Jupyter notebook from skipgram_batch_generator.ipynb


In [2]:
batch_size=32
learning_rate=0.5
window_size = 3
vector_len = 50
vocab_size = 33702
neg_sample=20
steps = 150000
epochs = 15

val_indexes = [445,633,4904,16842,1478]

def skipgram():
    Xs = tf.placeholder(tf.int32 ,[batch_size,])
    Ys = tf.placeholder(tf.int32 , [batch_size,1])
    
    with tf.variable_scope('skipgramWord2Vec') as scope:
        Vs = tf.Variable(tf.random_uniform([vocab_size , vector_len] , -1.0 , 1.0))
        Us = tf.Variable(tf.truncated_normal([vocab_size , vector_len] , stddev= 1.0/math.sqrt(vector_len)))
        
        embds = tf.nn.embedding_lookup(Vs , Xs)
        
        nce_biases = tf.Variable(tf.zeros([vocab_size]))
        
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=Us, 
                                        biases=nce_biases, 
                                        labels=Ys, 
                                        inputs=embds, 
                                        num_sampled=neg_sample, 
                                        num_classes=vocab_size), name='loss')
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(Vs), 1))
        normalized_Vs= tf.divide(Vs , tf.reshape(norm, (-1, 1)))
         
        val_embd = tf.identity(normalized_Vs)
        similarity = tf.matmul(val_embd , normalized_Vs , transpose_b=True)              
        
        
        return Xs , Ys , Vs , loss , similarity

In [3]:
def generator():
    yield from skipgram_batch_generator.gen_batch(window_size , batch_size)

In [4]:
def run():
    with open("indexWords.txt" , "rt") as infile:
        index_words = eval(infile.read())
        
    data = tf.data.Dataset.from_generator(generator, (tf.int32, tf.int32))
    
    iterator = data.make_initializable_iterator()
    
    centers, targets = iterator.get_next()    
    
    Xs , Ys , Vs , loss , similarity = skipgram()

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
   
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())
        for i in range (steps):
            try:
                batch_loss , _ = sess.run([loss , optimizer] , feed_dict={Xs: centers.eval() , Ys: targets.eval()})
                    
                if i%1000==0:
                    print(str(i)+":"+str(batch_loss))

                if i%5000 == 0:
                    sim = similarity.eval()

                    for j in val_indexes:
                        listt = (-sim[j,:]).argsort()[1:6]
                        print(listt)
                        print (index_words[j])
                        list2 = [index_words[k] for k in listt]
                        print (list2)



            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
                
       

In [5]:
if __name__=='__main__':
    run()

0:101.45906
[13759   912 21172  6502  8439]
two
['ller', 'central', 'hammerhead', 'raphael', 'martyrs']
[ 7277  2365 11361 32876 31992]
person
['dragon', 'fluctuations', 'poland', 'electropositive', 'nonsensical']
[ 1516   447 30527 32714 23800]
england
['caused', 'josiah', 'synth', 'tubing', 'gretel']
[15453 24719  9295 10992 13935]
week
['silence', 'conductors', 'chile', 'airborne', 'cdots']
[ 8790 20675  4857 30795 24817]
big
['staff', 'faiths', 'indelible', 'tu', 'disparaged']
1000:69.08902
2000:47.08097
3000:32.386234
4000:21.22713
5000:45.09574
[  95  446 4462 4634 3832]
two
['but', 'five', 'slave', 'lincoln', 'presidential']
[  26 4221  312 2446 1202]
person
['in', 'death', 'four', 'help', 'thus']
[  15 1516 7353 1264 2920]
england
['the', 'caused', 'anthem', 'website', 'u']
[15453 24719  9295 10992 13935]
week
['silence', 'conductors', 'chile', 'airborne', 'cdots']
[8790 4857 7240  908 1017]
big
['staff', 'indelible', 'economists', 'wing', 'another']
6000:45.77086
7000:16.63799

C:\Users\Fahimeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: generator 'gen_batch' raised StopIteration
  


48000:5.1689234
49000:8.956288
50000:2.2714217
[252 196 275 311 446]
two
['seven', 'one', 'nine', 'eight', 'five']
[ 1502  9169   613 10385  5548]
person
['rejection', 'debut', 'organisations', 'convert', 'alienate']
[ 1100  4537  1516   238 30527]
england
['god', 'ghost', 'caused', 'other', 'synth']
[10992   254 15751 20510   548]
week
['airborne', 'three', 'numerical', 'vector', 'currents']
[8790 9153 4857 1454  664]
big
['staff', 'headquarters', 'indelible', 'out', 'alexander']
51000:3.3588605
52000:5.8351965
53000:3.2536283
54000:4.310568
55000:4.5276566
[311 196 446 197 275]
two
['eight', 'one', 'five', 'six', 'nine']
[ 9169  5548  1502    26 10385]
person
['debut', 'alienate', 'rejection', 'in', 'convert']
[   24  1100  4537  1516 30527]
england
['is', 'god', 'ghost', 'caused', 'synth']
[10992 15751  5393 20510   548]
week
['airborne', 'numerical', 'wake', 'vector', 'currents']
[ 8790  9153  4857 16728  1454]
big
['staff', 'headquarters', 'indelible', 'colors', 'out']
56000:7.960

128000:2.639133
129000:3.6143026
130000:2.9116077
[254 312 196 197   5]
two
['three', 'four', 'one', 'six', 'of']
[   40  1502  2715  5548 19920]
person
['has', 'rejection', 'choosing', 'alienate', 'boers']
[ 4537 32714 12867   238  2920]
england
['ghost', 'tubing', 'paeans', 'other', 'u']
[10992 17144 24719   197   548]
week
['airborne', 'commend', 'conductors', 'six', 'currents']
[ 8790 25523  4857  9153   315]
big
['staff', 'muttered', 'indelible', 'headquarters', 'for']
131000:2.7528033
132000:3.3428385
133000:3.2944021
134000:2.4730942
135000:4.760549
[196 197 312 446 253]
two
['one', 'six', 'four', 'five', 'zero']
[   40  1502  2715  5548 26788]
person
['has', 'rejection', 'choosing', 'alienate', 'stares']
[ 4537 32714 12867 20151 27129]
england
['ghost', 'tubing', 'paeans', 'malaria', 'suffice']
[10992 17144 24719   548 15751]
week
['airborne', 'commend', 'conductors', 'currents', 'numerical']
[ 8790 25523  4857    53  9153]
big
['staff', 'muttered', 'indelible', 'from', 'headqu